In [1]:
! pip install owlready2

You should consider upgrading via the 'C:\Users\bordi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np

In [3]:
from owlready2 import *
onto_path.append("C:/Users/bordi/OneDrive/Desktop/Gangemi/Ontologia")
onto = get_ontology("UniOnt.owl")
onto.load()


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


get_ontology("http://www.semanticweb.org/francesca/ontologies/2022/6/uni#")

In [4]:
dul = get_ontology("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl")
dul.load()

get_ontology("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")

In [5]:
print(list(onto.classes()))

[UniOnt.ItalianPublicUniversity, UniOnt.Department, UniOnt.DegreeProgramme, UniOnt.EducationalActivity, UniOnt.AssessmentMethod, DUL.Place, UniOnt.AcademicField, UniOnt.AccessType, UniOnt.BachelorDegree, UniOnt.Course, UniOnt.Curriculum, UniOnt.Qualification, UniOnt.FinalTest, UniOnt.InternationalDegree, UniOnt.Internship, UniOnt.Laboratory, UniOnt.LimitedAccessDegree, UniOnt.OpenAccessDegree, UniOnt.MasterDegree, UniOnt.Seminar, UniOnt.SingleCycleDegree]


In [6]:
data = pd.read_csv("FinalData.csv", sep=",", encoding="utf-8")

In [7]:
data = data.to_dict("index")

In [8]:
for record in data.values():
    my_degree = onto.DegreeProgramme(record["DegreeProgramme"].split("/")[-1])
    if record["International"] is True:
        my_degree = onto.InternationalDegree(record["DegreeProgramme"].split("/")[-1])
        
#type       
    if record["type"] == "Laurea":
        my_degree = onto.BachelorDegree(record["DegreeProgramme"].split("/")[-1])
    if record["type"] == "Laurea Magistrale":
        my_degree = onto.MasterDegree(record["DegreeProgramme"].split("/")[-1])
    if record["type"] == "Laurea Magistrale a ciclo unico":
        my_degree = onto.SingleCycleDegree(record["DegreeProgramme"].split("/")[-1])
        
        
#access     
    if "libero accesso" in record["AccessType"]: 
        my_degree.hasAccessType = [onto.AccessType("OpenAccess")]
        my_degree = onto.OpenAccessDegree(my_degree)
               
    if "numero chiuso" in record["AccessType"]:
        my_degree.hasAccessType = [onto.AccessType("LimitedAccess")]
        my_degree = onto.LimitedAccessDegree(my_degree)
                
    my_degree.label = record["DegreeProgrammeLabel"]

#academic field
    my_academicField = onto.AcademicField(record["AcademicField"].replace(" ", "_"))
    my_academicField.label = record["AcademicField"]
    
    my_degree.hasAcademicField.append(my_academicField)
    
#italian university 
    my_university = onto.ItalianPublicUniversity(record["Italian University"].replace(" ", "_"))
    my_university.label = record["Italian University"]
    
    my_degree.isDeliveredBy = [onto.ItalianPublicUniversity(my_university)]
 


    if "Exam" in record["typeOfActivity"]: 
        my_educational_activity = onto.Course(record["EducationalActivity"].replace(" ", "_"))
        my_educational_activity.hasSsd = [record["Ssd"]]
        
    if "Internship" in record["typeOfActivity"]: 
        my_educational_activity = onto.Internship(record["EducationalActivity"].replace(" ", "_"))
    if "Seminar" in record["typeOfActivity"]: 
        my_educational_activity = onto.Seminar(record["EducationalActivity"].replace(" ", "_"))
    if "Laboratory" in record["typeOfActivity"]: 
        my_educational_activity = onto.Laboratory(record["EducationalActivity"].replace(" ", "_"))
    if "Final Test" in record["typeOfActivity"]: 
        my_educational_activity = onto.FinalTest(record["EducationalActivity"].replace(" ", "_"))
    
          
    my_educational_activity.label = [record["EducationalActivity"]]
    my_educational_activity.hasCfu = [record["Cfu"]]
    my_educational_activity.hasTeachingCode = [record["TeachingCode"]]
    
    
    my_degree.hasEducationalActivity.append(my_educational_activity)
    
    
    #Relazione con i dipartimenti
    
    my_department = onto.Department(record["Department"].replace(" ", "_"))
    my_department.label = record["Department"]
    
    my_degree.isManagedBy = [onto.Department(my_department)]

    #relazione tra dipartimenti e università 
    
    my_department.isPartOf.append(my_university)


#creiamo l'url
    my_degree.hasUrl = [record["DegreeProgramme"]] #data property    
    
#academic year    
    my_degree.hasAcademicYear = [record["academicYear"]] #data property
    
#degree class    
    my_degree.hasDegreeClass = [record["DegreeClass"]] #data property 

#yearsofDuration
    my_degree.hasYearsOfDuration = [record["hasYearsOfDuration"]] #data property 
    
#language
    my_degree.hasTeachingLanguage = [record["TeachingLanguage"]] #data property 
   

    #qualification 
    if "Titolo unico" in record["Qualification"]: 
        my_degree.releases = [onto.Qualification("SingleQualification")]               
    if "Titolo multiplo" in record["Qualification"]:
        my_degree.releases = [onto.Qualification("MultipleQualification")]
    if "Titolo congiunto" in record["Qualification"]:
        my_degree.releases = [onto.Qualification("JointQualification")]  

#places (campus)
    my_campus = dul.Place(record["campus"].replace(" ", "_"))
    my_campus.label = record["campus"]
                               
    my_degree.hasCampus = [dul.Place(my_campus)]
                                                           
#places (placesOfTeaching)
    my_teachingPlace = dul.Place(record["PlaceOfTeaching"].replace(" ", "_"))
    my_teachingPlace.label = record["PlaceOfTeaching"]
    
    my_degree.hasPlaceOfTeaching = [dul.Place(my_teachingPlace)]


    
    
                                      
    
    
    
    #ricordarsi di aggiungere sempre le labels quando si creano nuovi individui 

In [9]:
onto.save("Provola.owl")

Ricorda di provare ad avviare il Reasoner e farlo andare per ore per vedere se arriva a calcolare le relazioni inverse

# Da ricordare per la documentation
1. Alcune classi non sono state popolate per mancanza di dati. Un esempio è la classe "Assessment Method", popolate da instances che non sono state associate a individui esistenti nel database. Anche Curriculum non è stata popolata a causa della mancanza e della difficoltà di integrazione dei dati. 
2. Ci siamo limitate al database Unibo ma la nostra Ontologia è potenzialmente più potente
